In [1]:
import tensorflow as tf
from keras.layers import Input, Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D, concatenate, Activation
from keras.applications.mobilenet import MobileNet as Net

from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Input, merge
from keras.optimizers import Adam
import cv2
import numpy as np
import os
#from tqdm import tqdm
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import time
import glob

ALPHA = 0.7 # Triplet Loss Parameter

from keras.layers import Input,Lambda,subtract,GlobalMaxPooling2D,Dense,GlobalAveragePooling2D,concatenate,Activation
from keras.applications.xception import Xception as Net
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
from keras.models import Model

import matplotlib.pyplot as plt

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
import random as rn

np.random.seed(0)
tf.set_random_seed(3)

from sklearn.model_selection import cross_val_score

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def img_to_array(image):
    img = cv2.resize(image, (128, 128)) 
    img1=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x_train = np.array([img1])
    return x_train

In [3]:
def img_path_to_array(image_path):
    img1 = cv2.imread(image_path, 1)
    return img_to_array(img1)

In [4]:
def prepare_database_tr(location):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0].strip('NISDCC-_6g')
        #print (identity)
        database[identity] = img_path_to_array(file)

    return database

In [5]:
train_image_arrays = prepare_database_tr('/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/training/*.PNG')


In [6]:
triplet_str_tr=[]

t1= time.time()
for i in range(1,13):
    for ja in range(1,6):
        for k in range(23,52):
             for jn in range(1,6):
                    for jp in range(1,6):
                        #print (i,ja,k,jn,jp)
                        if (jp!=ja):
                            
                            if(i<10):
                                a='00'+ str(i)+'_00'+str(i)+'_00'+str(ja)
                                b='00'+ str(i) +'_00'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_00'+str(i)+'_00'+str(jn)
                            else:
                                a='0'+str(i)+'_0'+str(i)+'_00'+str(ja)
                                b='0'+str(i)+'_0'+str(i)+'_00'+str(jp)
                                c='0'+str(k)+'_0'+str(i)+'_00'+str(jn)
                            
                            x= (a,b,c)
                            triplet_str_tr.append(x)
                        
                    
                    
                    
t2=time.time()

print (t2-t1)


0.125808954239


In [7]:
anchor_tr=[]
positive_tr=[]
negative_tr=[]

for img_name in triplet_str_tr:
    anchor_tr.append(train_image_arrays[img_name[0]])
    positive_tr.append(train_image_arrays[img_name[1]])
    negative_tr.append(train_image_arrays[img_name[2]])

In [8]:
x_a_tr=np.array(anchor_tr).reshape((len(anchor_tr), 128*128))
x_p_tr=np.array(positive_tr).reshape((len(positive_tr),128*128))
x_n_tr=np.array(negative_tr).reshape((len(negative_tr),128*128))


In [9]:
x_c=np.vstack((x_a_tr,x_p_tr,x_n_tr))

## PCA_FEATURES

In [10]:
from sklearn.decomposition import PCA
pca = PCA(n_components=700)
pca.fit(x_c)

PCA(copy=True, iterated_power='auto', n_components=700, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [11]:
sum(pca.explained_variance_ratio_)

0.9204018049071174

In [12]:
#Getting PCA features
pca_anc_tr= pca.transform(x_a_tr)
pca_pos_tr= pca.transform(x_p_tr)
pca_neg_tr= pca.transform(x_n_tr)

In [14]:
#Creating Unique Identification for training set
#Creating Unique Identification for test set embedding
pos_feat=np.zeros((len(x_a_tr),700))
neg_feat=np.zeros((len(x_a_tr),700))

for i in range(0,len(x_a_tr)):
    pos_feat[i]= pca_anc_tr[i]-pca_pos_tr[i]
    neg_feat[i]= pca_anc_tr[i]-pca_neg_tr[i]

In [15]:
comb_arr=np.vstack((pos_feat,neg_feat))

In [31]:
#np.sum(pca.explained_variance_ratio_)

In [16]:
pos_Y = [0]*len(pos_feat)
#makes that no of ones as the len of the item given
neg_Y = [1]*len(neg_feat)
#rowwise appends the two arrays
train_y=np.append(pos_Y, neg_Y)

In [17]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(comb_arr, train_y, test_size=0.2, random_state=42)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(random_state=0)
log_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
log_model.score(x_test,y_test)

1.0

In [20]:
print (recall_score(y_test, log_model.predict(x_test)))
print (precision_score(y_test, log_model.predict(x_test)))

1.0
1.0


In [21]:
y_pred=log_model.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[6931,    0],
       [   0, 6989]])

In [22]:
image_path='/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/genuines/NFI-08402084.png'
#image_id = 'souradip'

In [23]:
input_img_org= img_path_to_array(image_path)

In [24]:
image_path1='/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/forgeries/NFI-04901084.png'
input_img_org2=img_path_to_array(image_path1)

In [28]:
img1=np.resize(input_img_org,(1,128*128))
img2=np.resize(input_img_org2,(1,128*128))

In [29]:
i1=pca.transform(img1)
i2=pca.transform(img2)

In [30]:
diff=i1-i2

In [31]:
log_model.predict(diff)

array([0])

In [32]:
log_model.predict_proba(diff)

array([[0.86498929, 0.13501071]])

# Random Forest

In [51]:
from sklearn.ensemble import RandomForestClassifier
forest_model = RandomForestClassifier(n_estimators=300, max_depth=10,random_state=0)

In [52]:
forest_model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [53]:
forest_model.score(x_test,y_test)

1.0

## Test Data

In [33]:
all_ids = np.arange(1, 101)
missing_ids = np.array([5, 13, 25, 32, 34, 36, 38, 40, 48, 50, 52, 57, 60, 61, 65, 76, 78, 81, 82, 87, 95])
ids_available = np.array(list((set(all_ids)-set(missing_ids))))


In [34]:
j_formatted=[]
for i in range(1,13):
    if (i<10):
        new_j='0'+str(i)
    else:
        new_j=str(i)
    j_formatted.append(new_j)

In [35]:
i_formatted=[]
for i in ids_available:
    if (i<10):
        new_i='00'+str(i)
    elif(9<i<100):
        new_i='0'+str(i)
    else:
        new_i= str(i)
    i_formatted.append(new_i)

In [36]:
f_formatted=[]
for i in range(3,101):
    if (i<10):
        new_f='00'+str(i)
    elif(9<i<100):
        new_f='0'+str(i)
    else:
        new_f= str(i)
    f_formatted.append(new_f)

In [37]:
triplet_test_comb=[]
for i in i_formatted:
    for f in f_formatted:
        for ja in j_formatted:
            for jn in j_formatted[0:6]:
                for jp in j_formatted:
                    if (jp!=ja):
                        a=i+ja+i
                        p=i+jp+i
                        n=f+jn+i
                        triplet_test=(a,p,n)
                        triplet_test_comb.append(triplet_test)

In [38]:
def prepare_test_dict(location):
    database = {}

    # load all the images of individuals to recognize into the database
    for file in glob.glob(location):
        identity = os.path.splitext(os.path.basename(file))[0].strip('NFI-')
        #print (identity)
        database[identity] = img_path_to_array(file)

    return database


In [39]:
test_genuine_dict = prepare_test_dict('/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/genuines/*')
test_forgery_dict = prepare_test_dict('/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/forgeries/*')


In [40]:
final_triplet_test_id=[]
for i in range(0,len(triplet_test_comb)):
    if ((triplet_test_comb[i][0] in test_genuine_dict.keys()) and (triplet_test_comb[i][1] in test_genuine_dict.keys())and (triplet_test_comb[i][2] in test_forgery_dict.keys())):
        final_triplet_test_id.append(triplet_test_comb[i])
        

In [41]:
anchor_test=[]
positive_test=[]
negative_test=[]

for img_name in final_triplet_test_id:
    anchor_test.append(test_genuine_dict[img_name[0]])
    positive_test.append(test_genuine_dict[img_name[1]])
    negative_test.append(test_forgery_dict[img_name[2]])

In [42]:
x_a_test=np.array(anchor_test).reshape((len(anchor_test), 128*128))
x_p_test=np.array(positive_test).reshape((len(positive_test),128*128))
x_n_test=np.array(negative_test).reshape((len(negative_test),128*128))


In [43]:
#Combine Text Features
x_c_test=np.vstack((x_a_test,x_p_test,x_n_test))

In [44]:
pca_anc_test= pca.transform(x_a_test)
pca_pos_test= pca.transform(x_p_test)
pca_neg_test= pca.transform(x_n_test)

In [626]:
#pca_anc_test.shape

In [46]:
pos_feat_test=np.zeros((len(pca_anc_test),700))
neg_feat_test=np.zeros((len(pca_anc_test),700))

for i in range(0,len(pca_anc_test)):
    pos_feat_test[i]= pca_anc_test[i]-pca_pos_test[i]
    neg_feat_test[i]= pca_anc_test[i]-pca_neg_test[i]

In [47]:
pos_Y_test = [0]*len(pos_feat_test)
#makes that no of ones as the len of the item given
neg_Y_test = [1]*len(neg_feat_test)
#rowwise appends the two arrays
test_y=np.append(pos_Y_test, neg_Y_test)

In [48]:
test_feat=np.vstack((pos_feat_test,neg_feat_test))

In [55]:
#forest_model.score(test_feat,test_y)

In [50]:
print (recall_score(test_y, log_model.predict(test_feat)))
print (precision_score(test_y, log_model.predict(test_feat)))

0.39822322322322323
0.5267286749859322


## HOG Features

In [155]:
x_a_tr=np.array(anchor_tr).reshape((len(anchor_tr), 200,200))
x_p_tr=np.array(positive_tr).reshape((len(positive_tr),200,200))
x_n_tr=np.array(negative_tr).reshape((len(negative_tr),200,200))


In [157]:
def get_hog_feature_array(image_list):
    hog_feat_arr=np.zeros((len(image_list),20736))
    from skimage import feature
    for i in range(0,len(image_list)):
        H = feature.hog(image_list[i], orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")
        hog_feat_arr[i]=H
        
    return hog_feat_arr

In [158]:
#Getting HOG Embeddings
x_a_hog= get_hog_feature_array(x_a_tr)
x_p_hog= get_hog_feature_array(x_p_tr)
x_n_hog= get_hog_feature_array(x_n_tr)

In [160]:
x_a_hog.shape

(34800, 20736)

In [161]:
#Creating Unique Identification for training set
#Creating Unique Identification for test set embedding
pos_feat_hog=np.zeros((len(x_a_tr),20736))
neg_feat_hog=np.zeros((len(x_n_tr),20736))

for i in range(0,len(x_a_tr)):
    pos_feat_hog[i]= x_a_hog[i]-x_p_hog[i]
    neg_feat_hog[i]= x_a_hog[i]-x_n_hog[i]

In [162]:
comb_arr_hog=np.vstack((pos_feat_hog,neg_feat_hog))

In [163]:
pos_Y_hog = [0]*len(pos_feat_hog)
#makes that no of ones as the len of the item given
neg_Y_hog = [1]*len(neg_feat_hog)
#rowwise appends the two arrays
train_y_hog=np.append(pos_Y_hog, neg_Y_hog)

In [164]:
from sklearn.cross_validation import train_test_split
x_train_hog, x_test_hog, y_train_hog, y_test_hog = train_test_split(comb_arr_hog, train_y_hog, test_size=0.2, random_state=42)


In [165]:
from sklearn.linear_model import LogisticRegression
log_model_hog = LogisticRegression(random_state=0)
log_model_hog.fit(x_train_hog,y_train_hog)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [166]:
log_model_hog.score(x_test_hog,y_test_hog)

1.0

In [167]:
print (recall_score(y_test_hog, log_model_hog.predict(x_test_hog)))
print (precision_score(y_test_hog, log_model_hog.predict(x_test_hog)))

1.0
1.0


In [545]:
image_path='/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/genuines/NFI-08402084.png'
#image_id = 'souradip'

In [278]:
input_img_org= img_path_to_array(image_path)

In [279]:
image_path1='/Users/s0c02nj/Desktop/Dataset-AxisBankAI/final_database_sign/test/forgeries/NFI-00304046.png'
input_img_org2=img_path_to_array(image_path1)

In [280]:
img1=np.resize(input_img_org,(200,200))
img2=np.resize(input_img_org2,(200,200))

In [281]:
from skimage import feature
i1 = feature.hog(img1, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")

i2= feature.hog(img2, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), transform_sqrt=True, block_norm="L2")

In [282]:
i1.resize(1,len(i1))
i2.resize(1,len(i2))

In [283]:
diff=i1-i2
#diff.shape

In [284]:
log_model_hog.predict(diff)

array([0])

In [285]:
log_model_hog.predict_proba(diff)

array([[0.9871637, 0.0128363]])